In [3]:
from minio import Minio
#from minio.error import ResponseError

# Minio configuration
minio_endpoint = "localhost:9000"   # Replace with your Minio service endpoint


# minio_access_key = "bOtcUxnnC3TTbdZYxwkE"
# minio_secret_key = "Oq8udlsqAEJFQAiXybGd95iRa3Zlsv2xOmbuY9Xi"

minio_access_key = "minioadmin"
minio_secret_key = "minioadmin"

minio_secure = False  # Set to True if using HTTPS
# MLflow configuration
mlflow_bucket = "mlflow"

# Initialize Minio client
minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=minio_secure)

def check_minio_bucket(bucket_name):
    try:
        # Check if the bucket exists
        if not minio_client.bucket_exists(bucket_name):
            print(f"Bucket '{bucket_name}' does not exist.")
            return

        # List objects in the bucket
        objects = minio_client.list_objects(bucket_name)
        print(f"Objects in '{bucket_name}':")
        for obj in objects:
            print(f" - {obj.object_name}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    check_minio_bucket(mlflow_bucket)


Objects in 'mlflow':


## Create a new Bucket on MinIO

In [9]:
from minio import Minio
from minio.error import S3Error

# Replace with your Minio credentials and Minio service IP
minio_endpoint = "localhost:9000"  # Use the default API port (9000)
bucket_name = "mlflow"

minio_access_key = "minioadmin"
minio_secret_key = "minioadmin"

# Initialize Minio client
minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=False)

# Create a bucket
try:
    minio_client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except S3Error as e:
    if e.code == "BucketAlreadyOwnedByYou":
        print(f"Bucket '{bucket_name}' already exists.")
    else:
        print(f"Error creating bucket: {e}")


Bucket 'new-mlflow' created successfully.


In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import os
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# this needs to map the mlflow pod internal address 
# and not use the local MLFLOW_TRACKING_URI set by 
# the local mlflow process
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"

# minio_access_id = os.environ.get("AWS_ACCESS_KEY_ID")
# minio_secret_access_key = os.environ.get("AWS_SECRET_ACCESS_KEY") 
# print(minio_access_id,os.environ.get("AWS_SECRET_ACCESS_KEY"))

if __name__ == "__main__":
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.end_run()
    with mlflow.start_run():
        
        X = np.array([-1, 1, -1, 2, 3, 1, -1, 1, 0, 2, 3, 1, ]).reshape(-1, 1)
        y = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0])
        lr = LogisticRegression()
        lr.fit(X, y)
        score = lr.score(X, y)
        print(f"Score: {score}")
        mlflow.log_metric("score", score)
        predictions = lr.predict(X)
        signature = infer_signature(X, predictions)
        mlflow.sklearn.log_model(lr, "model")
        print(f"Model saved in run {mlflow.active_run().info.run_uuid}")

Score: 0.75
Model saved in run 32d0536867384f68abcb222562ccf980


/home/t2/micromamba/envs/env_5/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
